# **Большой Бот про всё (кроме ПОС)**
![alt text](big_bot-1.png)

[схематически](https://boards.yandex.ru/whiteboard/?hash=d9431dea9bcda9a491849e7e0af7d37e)

In [18]:
# библиотеки
import pandas as pd
import numpy as np
import os
import sys
import sqlite3
import telebot
from telebot import types
from telebot.types import (
    InlineKeyboardMarkup,
    InlineKeyboardButton,
    ReplyKeyboardRemove,
)
import datetime
import time
from datetime import datetime, timedelta
import random
from PIL import Image, ImageDraw, ImageFont
import xlwings as xw
import math

pd.options.mode.chained_assignment = None  # default='warn'

# пути
dir_path = os.getcwd().replace("\\", "/")
e_path = "/".join([dir_path, "source", "e"]) + "/"
s_path = "/".join([dir_path, "source", "s"]) + "/"
dm_path = "/".join([dir_path, "source", "dm"]) + "/"
dm_etalon_path = "/".join([dir_path, "source", "dm_etalon"]) + "/"
db_path = "/".join([dir_path, "pb_db.sqlite"])
img_path = "/".join([dir_path, "img/"])


def transponder():
    """
    Опознание нового юзера: в первой и второй паре цифр пин-кода из большего значение вычесть меньшее, результаты перемножить.
    """
    question = [random.randint(0, 9) for _ in range(4)]
    answer = abs(question[0] - question[1]) * abs(question[2] - question[3])
    return question, answer


def user_db(user_id, date_time, flag):
    """
    Регистрация нового юзера в БД
    """
    global db_path
    query_authorization_reselt = """
    INSERT INTO user_status (user_id, date_time, flag) VALUES (?, ?, ?);
    """
    try:
        conn = sqlite3.connect(db_path)
        cur = conn.cursor()
        cur.execute(
            query_authorization_reselt,
            (
                user_id,
                date_time,
                flag,
            ),
        )
        conn.commit()
        return True, ""
    except Exception as e:
        return False, e


def datastatus(e_path, s_path, dm_path, dm_etalon_path, db_path):
    """
    Загрузка исходников и определение статуса данных, возможности расчётов
    """
    # ошибки
    e_path
    e_filename_lst = os.listdir(e_path)
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_error_type = """
    SELECT et.final_state, et.error_type, et.error_type_dhb
    FROM error_type et;
    """
    error_type_query = cur.execute(query_error_type).fetchall()
    df_error_type = pd.DataFrame(
        error_type_query, columns=["final_state", "error_type", "error_type_dhb"]
    )
    etalon_final_state_list = df_error_type["final_state"].tolist()

    if len(e_filename_lst) != 1:
        e_file_status = """
    📙 ОШИБКИ ЗАПИСИ
        🔸в целевой директории нет файла или более одного файла
    """
    else:
        e_filename = e_filename_lst[0]
        e_file_path = "".join([e_path, e_filename])
        e_file_modification = datetime.fromtimestamp(
            os.path.getmtime(e_file_path)
        ).strftime("%Y-%m-%d %H:%M:%S")

        df_e = pd.read_csv(e_file_path, sep=";")
        df_e = df_e[df_e["Тип записи"] == 3]
        df_e = df_e.merge(df_error_type, on="final_state", how="left")
        df_e["create_ts"] = pd.to_datetime(df_e["create_ts"], format="mixed")
        df_e["create_ts"] = df_e["create_ts"].apply(
            lambda x: x.replace(tzinfo=None).date()
        )
        e_date_min = df_e["create_ts"].min().strftime("%Y-%m-%d")
        e_date_max = df_e["create_ts"].max().strftime("%Y-%m-%d")

        # критические пропуски данных
        df_nonan = df_e[
            [
                "session_id",
                "Название субъекта РФ",
                "step_id",
                "step_name",
                "error_code",
                "final_state",
                "create_ts",
                "Тип записи",
                "referral_flg",
            ]
        ]
        n = 0
        str_0 = "🔸критические пропуски данных:"
        for column in df_nonan.columns:
            if df_nonan[column].isnull().sum() != 0:
                next_str = f"\n\t- столбец '{column}': {df_nonan[column].isnull().sum()} ({round(df_nonan[column].isnull().sum()*100/df_nonan.shape[0], 2)} %)"
                str_0 += next_str
                n += 1
        if n == 0:
            str_0 += " отсутствуют"

        # ошибки данных
        str_1 = "🔸нарушение полноты данных: всего (доля от числа строк на соответствующем этапе):"

        # число строк на соответствующих этапах
        len_ValidatePerson = df_e[df_e["step_name"] == "ValidatePerson"].shape[0]
        len_MO = df_e[df_e["step_name"] == "MO"].shape[0]
        len_ServiceOrSpecs = df_e[df_e["step_name"] == "ServiceOrSpecs"].shape[0]
        len_Slot = df_e[df_e["step_name"] == "Slot"].shape[0]
        len_Book = df_e[df_e["step_name"] == "Book"].shape[0]

        """
        На этапе валидации пациента
        есть данные oid МО
        ИЛИ есть данные о названии МО
        ИЛИ есть данные о специальности врача
        ИЛИ есть данные о слоте
        """
        df_ValidatePerson = df_e[
            (df_e["step_name"] == "ValidatePerson")
            & (
                df_e["mo_oid"].notnull()
                | df_e["mo_name"].notnull()
                | df_e["Post_name"].notnull()
                | df_e["slot_ts"].notnull()
            )
        ]
        """
        На этапе выбора МО
        есть данные oid МО
        ИЛИ есть данные о названии МО
        ИЛИ есть данные о специальности врача
        ИЛИ есть данные о слоте
        """
        df_MO = df_e[
            (df_e["step_name"] == "MO")
            & (
                df_e["mo_oid"].notnull()
                | df_e["mo_name"].notnull()
                | df_e["Post_name"].notnull()
                | df_e["slot_ts"].notnull()
            )
        ]
        """
        На этапе выбора специальности врача
        нет данных об oid МО
        ИЛИ нет данных о названии МО
        ИЛИ есть данные о специальности врача
        ИЛИ есть данные о слоте
        """
        df_ServiceOrSpecs = df_e[
            (df_e["step_name"] == "ServiceOrSpecs")
            & (
                ~df_e["mo_oid"].notnull()
                | ~df_e["mo_name"].notnull()
                | df_e["Post_name"].notnull()
                | df_e["slot_ts"].notnull()
            )
        ]
        """"
        На этапе выбора слота
        нет данных об oid МО
        ИЛИ нет данных о названии МО
        ИЛИ нет данных о специальтности врача
        ИЛИ есть данные о времени слота
        """
        df_Slot = df_e[
            (df_e["step_name"] == "Slot")
            & (
                ~df_e["mo_oid"].notnull()
                | ~df_e["mo_name"].notnull()
                | ~df_e["Post_name"].notnull()
                | df_e["slot_ts"].notnull()
            )
        ]
        """
        На этапе записи
        нет любых данных (ИЛИ)
        """
        df_Book = df_e[
            (df_e["step_name"] == "Book")
            & (
                ~df_e["mo_oid"].notnull()
                | ~df_e["mo_name"].notnull()
                | ~df_e["Post_name"].notnull()
                | ~df_e["slot_ts"].notnull()
            )
        ]

        if df_ValidatePerson.shape[0] > 0:
            str_1 += f"\n\t- ValidatePerson: {df_ValidatePerson.shape[0]} ({round(df_ValidatePerson.shape[0]*100/len_ValidatePerson, 2)} %)"
        if df_MO.shape[0] > 0:
            str_1 += (
                f"\n\t- MO: {df_MO.shape[0]} ({round(df_MO.shape[0]*100/len_MO, 2)} %)"
            )
        if df_ServiceOrSpecs.shape[0] > 0:
            str_1 += f"\n\t- ServiceOrSpecs: {df_ServiceOrSpecs.shape[0]} ({round(df_ServiceOrSpecs.shape[0]*100/len_ServiceOrSpecs, 2)} %)"
        if df_Slot.shape[0] > 0:
            str_1 += f"\n\t- Slot: {df_Slot.shape[0]} ({round(df_Slot.shape[0]*100/len_Slot, 2)} %)"
        if df_Book.shape[0] > 0:
            str_1 += f"\n\t- Book: {df_Book.shape[0]} ({round(df_Book.shape[0]*100/len_Book, 2)} %)"
        if (
            df_ValidatePerson.shape[0]
            + df_MO.shape[0]
            + df_ServiceOrSpecs.shape[0]
            + df_Slot.shape[0]
            + df_Book.shape[0]
            == 0
        ):
            str_1 += " отсутствуют"

        # нераспознанные final_state
        str_2 = "🔸нераспознанные final_state:"
        final_state_list = df_e["final_state"].tolist()
        unnow_final_state_list = list(
            set([x for x in final_state_list if x not in etalon_final_state_list])
        )
        if len(unnow_final_state_list) > 0:
            for i in range(0, len(unnow_final_state_list)):
                str_2 += f"\n\t- {unnow_final_state_list[i]}"
        else:
            str_2 += " отсутствуют"

        # при подготовке данных в фрейм добавляется одни столбец (error_type) и их становится 14
        e_file_status = f"""
    📙 ОШИБКИ ЗАПИСИ
    🔹файл: {e_filename}
    🔹сохранён: {e_file_modification}
    🔹дата данных: {e_date_max} ({e_date_min})
    🔹число столбцов: {df_e.shape[1] - 2} ({"верное" if df_e.shape[1] == 15 else "неверное"})
    🔹число строк: {df_e.shape[0]}
    {str_0}
    {str_1}
    {str_2}
    """

    # успехи
    s_path
    s_filename_lst = os.listdir(s_path)

    if len(s_filename_lst) != 1:
        s_file_status = """
    📗 УСПЕШНЫЕ СЕССИИ
        🔸в целевой директории нет файла или более одного файла
    """
    else:
        s_filename = s_filename_lst[0]
        s_file_path = "".join([s_path, s_filename])
        s_file_modification = datetime.fromtimestamp(
            os.path.getmtime(s_file_path)
        ).strftime("%Y-%m-%d %H:%M:%S")

        df_s = pd.read_csv(s_file_path, sep=";")
        df_s = df_s[df_s["Тип записи"] == 3]
        df_s["Дата"] = pd.to_datetime(df_s["Дата"], format="mixed")
        df_s["slot_ts"] = pd.to_datetime(df_s["slot_ts"], format="mixed")
        df_s["create_ts"] = pd.to_datetime(df_s["create_ts"], format="mixed")
        df_s["slot_ts"] = df_s["slot_ts"].apply(lambda x: x.replace(tzinfo=None))
        df_s["create_ts"] = df_s["create_ts"].apply(lambda x: x.replace(tzinfo=None))
        s_date_min = df_s["Дата"].min().strftime("%Y-%m-%d")
        s_date_max = df_s["Дата"].max().strftime("%Y-%m-%d")

        # пропуски данных
        missing_s_dict = {}
        for column in df_s.columns:
            missing_s_dict[column] = [
                df_s[column].isnull().sum(),
                round(df_s[column].isnull().sum() * 100 / df_s.shape[0], 2),
            ]
        str_0 = "🔸невалидные пропуски данных: всего (доля от общего числа строк):"
        missing_count = 0
        for k in missing_s_dict.keys():
            if missing_s_dict.get(k)[0] != 0:
                str_0 += f"\n\t- {k}: {missing_s_dict.get(k)[0]} ({missing_s_dict.get(k)[1]} %)"
                missing_count += missing_s_dict.get(k)[0]
        if missing_count == 0:
            str_0 += " отсутствуют"

        s_file_status = f"""
    📗 УСПЕШНЫЕ СЕССИИ
    🔹файл: {s_filename}
    🔹сохранён: {s_file_modification}
    🔹дата данных: {s_date_max} ({s_date_min})
    🔹число столбцов: {df_s.shape[1]} ({"верное" if df_s.shape[1] == 9 else "неверное"})
    🔹число строк: {df_s.shape[0]}
    {str_0}
    """

    # витрины
    dm_path
    dm_etalon_path
    dm_filename_lst = os.listdir(dm_path)
    dm_etalon_filename_lst = os.listdir(dm_etalon_path)

    if len(dm_filename_lst) != 1:
        dm_file_status = """
    📘 ВИТРИНЫ ДАННЫХ
        🔸в целевой директории нет файла или более одного файла
    """
    else:
        dm_filename = dm_filename_lst[0]
        dm_file_path = "".join([dm_path, dm_filename])
        dm_file_modification = datetime.fromtimestamp(
            os.path.getmtime(dm_file_path)
        ).strftime("%Y-%m-%d %H:%M:%S")

        df_dm = pd.read_excel(
            dm_file_path,
            sheet_name="Sheet1",
            index_col=None,
        )

        region_count = df_dm["Регион"].nunique()

        if len(dm_etalon_filename_lst) != 1:
            etalon = "🔹эталонного файла витрин нет или в целевой директории более одного файла"
            df_dm_etalon = pd.DataFrame()
        else:
            dm_etalon_filename = dm_etalon_filename_lst[0]
            dm_etalon_file_path = "".join([dm_etalon_path, dm_etalon_filename])
            dm_etalon_file_modification = datetime.fromtimestamp(
                os.path.getmtime(dm_etalon_file_path)
            ).strftime("%Y-%m-%d %H:%M:%S")

            df_dm_etalon = pd.read_excel(
                dm_etalon_file_path,
                sheet_name="Sheet1",
                index_col=None,
            )
            etalon_region_count = df_dm_etalon[
                df_dm_etalon["Статус витрины"] != "Не внедрена"
            ]["Регион"].nunique()

        if df_dm_etalon.shape[0] == 0:
            df_dm
        else:
            df_dm = df_dm.merge(
                pd.DataFrame(
                    df_dm_etalon[df_dm_etalon["Статус витрины"] != "Не внедрена"][
                        "Регион"
                    ].unique()
                ),
                left_on="Регион",
                right_on=0,
                how="inner",
            )
            df_dm = df_dm[[col for col in df_dm.columns[:-1]]]

            etalon = f""" 🔸эталонный файл: {dm_etalon_filename}
     🔸сохранён: {dm_etalon_file_modification}
     🔸количество витринных регионов в эталонном файле: {etalon_region_count}
     🔸количество витринных регионов будет в рассылке: {df_dm["Регион"].nunique()}"""

        df_dm["Период"] = pd.to_datetime(df_dm["Период"], format="%d.%m.%Y")
        dm_date_min = df_dm["Период"].min().strftime("%Y-%m-%d")
        dm_date_max = df_dm["Период"].max().strftime("%Y-%m-%d")

        # проверка на отрицательные значения
        str_0 = "🔸отрицательные значения в валидных столбцах:"
        minus = 0
        for col in df_dm.columns:
            try:
                if df_dm[col].min() < 0:
                    str_0 += f"\n\t- {col}"
                    minus += 1
            except Exception:
                pass
        if minus == 0:
            str_0 += " отсутствуют (без учёта стобцов, в которых могут быть нечисловые значения)"

        # проверка на нечисловые значения
        str_1 = "🔸нечисловые значения в валидных столбцах:"
        non_numeric = 0
        for col in df_dm.columns[5:]:
            try:
                if df_dm[col].sum() >= 0:
                    pass
            except Exception:
                str_1 += f"\n\t- {col}"
                non_numeric += 1
        if non_numeric == 0:
            str_1 += " отсутствуют"

        dm_file_status = f"""
    📘 ВИТРИНЫ ДАННЫХ
    🔹файл: {dm_filename}
    🔹сохранён: {dm_file_modification}
    🔹количество витринных регинов в основном файле: {region_count}
    {etalon}
    🔹дата данных: {dm_date_max} ({dm_date_min})
    🔹число столбцов: {df_dm.shape[1]} ({"верное" if df_dm.shape[1] == 13 else "неверное"})
    🔹число строк: {df_s.shape[0]}
    {str_1}
    {str_0}
    """

        if (
            len(
                set(
                    [
                        e_date_max,
                        e_date_min,
                        s_date_max,
                        s_date_min,
                        dm_date_max,
                        dm_date_min,
                    ]
                )
            )
            > 1
        ):
            fail = True
            result_str_0 = f"""
    🔴Даты данных не совпадают, дальнейший расчёт невозможен!
        """
        else:
            fail = False
            result_str_0 = f"""
    🟢Даты всех наборов данных корректны, совпадают.
        """
        if non_numeric != 0:
            fail = True
            result_str_1 = f"""
    🔴В валидных столбцах файла витрин присутствуют нечисловые значения, дальнейший расчёт невозможен!
        """
        else:
            fail = False
            result_str_1 = f"""
    🟢В валидных столбцах файла витрин нечисловых значений нет.
        """

    # даты из БД
    query = """
    SELECT DISTINCT date(dr.date) FROM data_rus dr
    """
    date_db = cur.execute(query).fetchall()
    date_db_list = [x[0] for x in date_db]
    # date_data = df_rus.loc[0, "date"].strftime("%Y-%m-%d")
    date_db_list
    try:
        status_db_date = f"""
        📌 Важно: даты данных, имеющихся в БД: {date_db_list[0]}, {date_db_list[1]}.
        """
    except Exception as e:
        status_db_date = f"""
        🔴 Важно: ошибка при получении дат из БД: {str(e)}.
        """

    status = (
        s_file_status
        + e_file_status
        + dm_file_status
        + result_str_0
        + result_str_1
        + status_db_date
    )

    return df_e, df_s, df_dm, status, fail


def reg_ref(db_path):
    """Возвращает полный справочник регионов (сначала неэталонное написание)"""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_regions = """
    SELECT r.reg_name_rus AS reg_name, r.reg_name_cm AS reg_name_data, public_flag
    FROM regions r
    UNION
    SELECT r.reg_name_rus AS reg_name, r.reg_name_mc AS reg_name_data, public_flag
    FROM regions r
    UNION
    SELECT r.reg_name_rus AS reg_name, r.reg_name_pos AS reg_name_data, public_flag
    FROM regions r
    UNION
    SELECT r.reg_name_rus AS reg_name, r.reg_name_rus AS reg_name_data, public_flag
    FROM regions r;
    """
    regions = cur.execute(query_regions).fetchall()
    df_regions = pd.DataFrame(
        regions, columns=["reg_name", "reg_name_data", "public_flag"]
    )
    return df_regions


def admins(db_path):
    """Возвращает список админов этого вот всего"""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_regions = """
    SELECT DISTINCT a.admin_id
    FROM admins a;
    """
    return list(cur.execute(query_regions).fetchall()[0])


def edit_db(query, db_path):
    """Обращение к БД"""
    try:
        conn = sqlite3.connect(db_path)
        cur = conn.cursor()
        # cur.execute(query)
        query_result = cur.execute(query).fetchall()
        conn.commit()
        return str(query_result)
    except Exception:
        return "🐏"


def dhb_status(df_e, df_s, db_path):
    """Сверка с дашбордом МЗ РФ"""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    send_region_query = """
    SELECT rsd.reg_name
    FROM reg_send_data rsd
    WHERE rsd.send_flag = 1
    """
    send_region_list = [x[0] for x in cur.execute(send_region_query).fetchall()]
    df_reg_ref = reg_ref(db_path)
    df_reg_ref = df_reg_ref[df_reg_ref["reg_name"].isin(send_region_list)]
    # выбор рандомного региона
    random_region_list = df_reg_ref.iloc[
        random.randint(0, df_reg_ref.shape[0])
    ].tolist()[:2]
    df_nodhb_reg = df_reg_ref[df_reg_ref["public_flag"] == 0]
    nodhb_reg_list = list(
        set(df_nodhb_reg["reg_name"].tolist() + df_nodhb_reg["reg_name_data"].tolist())
    )
    df_e_0 = df_e[
        (df_e["error_type_dhb"].notnull())
        & (~df_e["Название субъекта РФ"].isin(nodhb_reg_list))
    ]
    # успехи
    df_s_0 = df_s[~df_s["Название субъекта РФ"].isin(nodhb_reg_list)]
    dhb_status = f"""
    📆 СВЕРКА С ДАШБОРДОМ МИНЗДРАВА РОССИИ
    🇷🇺 Российская Федерация:
    🔹успешных записей: {df_s_0.shape[0]}
    🔹 попыток записи, прерванных пользователем: {df_e_0[df_e_0["error_type_dhb"] == "d"].shape[0]}
      ♦️ неудачных попыток записи:
    🔸по организационным причинам в субъектах: {df_e_0[df_e_0["error_type_dhb"] == "o"].shape[0]}
    🔸из-за ошибок федерального уровня: {df_e_0[df_e_0["error_type_dhb"] == "r"].shape[0]}
    🔸по техническим причинам в субъектах: {df_e_0[df_e_0["error_type_dhb"] == "t"].shape[0]}
    🏘 Субъект Российской Федерации (определяется случайным выбором):
    🔹регион: {random_region_list[0]}
    🔹успешных записей: {df_s_0[df_s_0["Название субъекта РФ"].isin(random_region_list)].shape[0]}
    🔹 попыток записи, прерванных пользователем: {df_e_0[(df_e_0["error_type_dhb"] == "d")&(df_e_0["Название субъекта РФ"].isin(random_region_list))].shape[0]}
      ♦️ неудачных попыток записи:
    🔸по организационным причинам в субъектах: {df_e_0[(df_e_0["error_type_dhb"] == "o")&(df_e_0["Название субъекта РФ"].isin(random_region_list))].shape[0]}
    🔸из-за ошибок федерального уровня: {df_e_0[(df_e_0["error_type_dhb"] == "r")&(df_e_0["Название субъекта РФ"].isin(random_region_list))].shape[0]}
    🔸по техническим причинам в субъектах: {df_e_0[(df_e_0["error_type_dhb"] == "t")&(df_e_0["Название субъекта РФ"].isin(random_region_list))].shape[0]}
    """
    return dhb_status


def img_filling(db_path, img_source_path):
    """
    Заливка шаблонов картинок в БД. Это разовая акция, для дальнейшей работы чат-бота функция сия не нужна.
    img_source_path должен непременно оканчиваться слэшем.
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_img_filling = """INSERT INTO images (id, image) VALUES (?, ?)"""

    for img in os.listdir(img_source_path):
        id = img
        i_path = "".join([img_source_path, img])
        with open(i_path, "rb") as file:
            blob_data = file.read()
        data = (id, blob_data)
        cur.execute(query_img_filling, data)
        conn.commit()
    # cur.close()


def img_make(img_path, db_path, imd_date):
    """
    Деление картинок для сообщений хождения по врачам. Агрументы: путь в папку картинок (в ней должна быть папка со шрифтом),
    путь к БД, дата в формате yyyy-mm-dd (это, сука, важно)
    """
    month_dict = {
        "01": "января",
        "02": "февраля",
        "03": "марта",
        "04": "апреля",
        "05": "мая",
        "06": "июня",
        "07": "июля",
        "08": "августа",
        "09": "сентября",
        "10": "октября",
        "11": "ноября",
        "12": "декабря",
    }
    img_date_str = " ".join(
        [
            imd_date.split("-")[2],
            month_dict.get(imd_date.split("-")[1]),
            imd_date.split("-")[0],
        ]
    )
    font_path = "".join([img_path, "font/Montserrat.ttf"])
    font_date = ImageFont.truetype(font_path, size=50)
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_img = """
    SELECT i.id, i.image
    FROM images i
    """
    images = cur.execute(query_img).fetchall()
    for row in images:
        img_name = row[0]
        img_save_path = "".join([img_path, img_name])
        img = row[1]
        with open(img_save_path, "wb") as file:
            file.write(img)
        img = Image.open(img_save_path)
        draw_text = ImageDraw.Draw(img)
        draw_text.text((135, 880), img_date_str, fill=("#444444"), font=font_date)
        img.save(img_save_path)
        img.close()


def calc_filling_rus(df_e, df_s, df_dm, db_path):
    """
    Расчёт и заливка в БД данных для федеральной рассылки. df_e, df_s, df_dm получены из datastatus, т.е.:
    - отфильтрованы по '3';
    - из dm убраны регионы с невнедрённой витриной.
    В качестве даты использована дата нулевой строки из df_dm.
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    df_reg_ref = reg_ref(db_path)

    # корректные названия регионов
    df_e_rr = df_e.merge(
        df_reg_ref, left_on="Название субъекта РФ", right_on="reg_name_data", how="left"
    ).drop("reg_name_data", axis=1)
    df_s_rr = df_s.merge(
        df_reg_ref, left_on="Название субъекта РФ", right_on="reg_name_data", how="left"
    ).drop("reg_name_data", axis=1)
    df_dm_rr = df_dm.merge(
        df_reg_ref, left_on="Регион", right_on="reg_name_data", how="left"
    ).drop("reg_name_data", axis=1)

    # КУ ФЭР
    # ось для резутьтатов - ПРАВИЛЬНАЯ, ИЗ БД
    query = """
    SELECT DISTINCT r.reg_name_rus, r.public_flag FROM regions r
    """
    df_axis = pd.DataFrame(
        cur.execute(query).fetchall(), columns=["reg_name", "public_flag"]
    ).drop_duplicates()
    df_axis["date"] = df_dm_rr.loc[0, "Период"]

    # успешные записи с учётом недашбордных регионов
    df_s_c = (
        pd.DataFrame(df_s_rr.groupby("reg_name")["Тип записи"].count())
        .reset_index()
        .rename(columns={"Тип записи": "s"})
    )

    # ошибки записи с учётом недашбордных регионов
    df_e_ct = (
        pd.DataFrame(
            df_e_rr[df_e_rr["error_type"] == "t"]
            .groupby("reg_name")["error_type"]
            .count()
        )
        .reset_index()
        .rename(columns={"error_type": "t"})
    )
    df_e_co = (
        pd.DataFrame(
            df_e_rr[df_e_rr["error_type"] == "o"]
            .groupby("reg_name")["error_type"]
            .count()
        )
        .reset_index()
        .rename(columns={"error_type": "o"})
    )
    df_e_cr = (
        pd.DataFrame(
            df_e_rr[df_e_rr["error_type"] == "r"]
            .groupby("reg_name")["error_type"]
            .count()
        )
        .reset_index()
        .rename(columns={"error_type": "r"})
    )

    # итог КУФЭР
    df_kufer_rus = (
        df_axis.merge(df_s_c, on="reg_name", how="left")
        .merge(df_e_co, on="reg_name", how="left")
        .merge(df_e_ct, on="reg_name", how="left")
        .merge(df_e_cr, on="reg_name", how="left")
        .fillna(0)
    )

    # Витрины
    df_dm_s = (
        pd.DataFrame(
            df_dm_rr[df_dm_rr["Шаг"] == df_dm_rr["Шаг"].unique().tolist()[3]]
            .groupby("reg_name")["Успешно"]
            .sum()
        )
        .reset_index()
        .rename(columns={"Успешно": "dm_s"})
    )
    df_dm_s.insert(1, "dm_impl", 1)
    df_dm_o = (
        pd.DataFrame(df_dm_rr.groupby("reg_name")["Орг. ошибки"].sum())
        .reset_index()
        .rename(columns={"Орг. ошибки": "dm_o"})
    )
    df_dm_t = (
        pd.DataFrame(df_dm_rr.groupby("reg_name")["Тех. ошибки"].sum())
        .reset_index()
        .rename(columns={"Тех. ошибки": "dm_t"})
    )
    df_dm_r = (
        pd.DataFrame(
            df_dm_rr.groupby("reg_name")["ЕПГУ ошибки"].sum()
            + df_dm_rr.groupby("reg_name")["ПОДД ошибки"].sum()
        )
        .reset_index()
        .rename(columns={0: "dm_r"})
    )

    # Итоги за день
    df_rus = (
        df_kufer_rus.merge(df_dm_s, on="reg_name", how="left")
        .merge(df_dm_o, on="reg_name", how="left")
        .merge(df_dm_t, on="reg_name", how="left")
        .merge(df_dm_r, on="reg_name", how="left")
        .fillna(0)
    )
    df_rus = df_rus.astype(
        {
            "public_flag": int,
            "s": int,
            "o": int,
            "t": int,
            "r": int,
            "dm_impl": int,
            "dm_s": int,
            "dm_o": int,
            "dm_t": int,
            "dm_r": int,
        }
    )
    # заливка в db
    query = """
    SELECT DISTINCT date(dr.date) FROM data_rus dr
    """
    date_db = cur.execute(query).fetchall()
    date_db_list = [x[0] for x in date_db]
    date_data = df_rus.loc[0, "date"].strftime("%Y-%m-%d")

    if date_data in date_db_list:
        result = f"❓Расчёт федеральной рассылки выполнен, но данные с датой {date_data} уже есть в БД."
    else:
        df_rus.to_sql("data_rus", conn, if_exists="append", index=False)
        # удаление данных старше, чем текущение - 1 день
        min_db_date = df_rus.loc[0, "date"] - timedelta(days=1)
        min_db_date_query = min_db_date.strftime("%Y-%m-%d")
        query_del_date = """
        DELETE FROM data_rus WHERE date < ?
        """
        min_db_date_query
        cur.execute(query_del_date, (min_db_date_query,))
        conn.commit()
        result = (
            f"👍Расчёт федеральной рассылки выполнен, данные на {date_data} залиты в БД"
        )
    return result, date_data


def rus_messages(dir_path, db_path):
    """
    Формирование федеральной рассылки (сообщения и файлы excel + pdf)
    """
    excel_path = "/".join([dir_path, "files/tables.xlsx"])
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_rus = """
    SELECT *
    FROM data_rus dr;
    """
    df_rus_calc = pd.DataFrame(
        cur.execute(query_rus).fetchall(),
        columns=[
            "reg_name",
            "public_flag",
            "date",
            "s",
            "o",
            "t",
            "r",
            "dm_impl",
            "dm_s",
            "dm_o",
            "dm_t",
            "dm_r",
        ],
    )

    df_rus_calc["date"] = pd.to_datetime(
        df_rus_calc["date"], format="%Y-%m-%d %H:%M:%S"
    )
    date_list = df_rus_calc["date"].unique().tolist()
    df_rus_last = df_rus_calc[
        (df_rus_calc["date"] == max(date_list))
        & (df_rus_calc["public_flag"] == 1)
        & (df_rus_calc["reg_name"] != "г. Москва")
    ]
    df_rus_last["sorted"] = df_rus_last.apply(
        lambda row: (
            1
            if (
                (row[4] + row[5] + row[9] + row[10]) == 0
                or (row[3] + row[4] + row[5] + row[8] + row[9] + row[10]) == 0
            )
            else (row[3] + row[8])
            / (row[3] + row[4] + row[5] + row[8] + row[9] + row[10])
        ),
        axis=1,
    )
    df_rus_last = df_rus_last.sort_values(by=["sorted", "s", "dm_s"], ascending=False)

    # перенос данных из df_rus_last в excel
    wb = xw.Book(excel_path)
    app = wb.app
    # выбор листа для вставки данных
    sht = wb.sheets["data"]
    # очистка диапазона
    sht.range("A:O").clear_contents()
    # вставка данных из DataFrame в ячейку A1
    sht.range("A1").value = df_rus_last
    # сохранение изменений
    wb.save()
    sht_pdf = wb.sheets["TAB"]
    sht_pdf.page_setup.print_area = "$A$1:$V$90"
    pdf_path = "".join(
        [
            dir_path + "/",
            "/files/",
            "rus_report_",
            max(date_list).strftime("%Y-%m-%d"),
            ".pdf",
        ]
    )
    sht_pdf.to_pdf(path=pdf_path, show=False)
    # сохранение изменений и окончание этого вот всего
    wb.save()
    wb.close()
    app.kill()

    # сообщение 1
    rus_0_stoplist = [
        "Донецкая Народная Республика",
        "Запорожская область",
        "Луганская Народная республика",
        "Херсонская область",
        "Харьковская область",
    ]
    df_rus_m0_0 = df_rus_calc[
        (df_rus_calc["date"] == max(date_list))
        & (~df_rus_calc["reg_name"].isin(rus_0_stoplist))
    ]
    df_rus_m0_1 = df_rus_calc[
        (df_rus_calc["date"] == min(date_list))
        & (~df_rus_calc["reg_name"].isin(rus_0_stoplist))
    ]

    def s_val(v_0, v_1):
        if v_0 > v_1:
            return "+" + str(v_0 - v_1)
        elif v_0 < v_1:
            return "-" + str(v_1 - v_0)
        else:
            return "без изменений"

    # отчётные сутки
    s_count_0 = (
        df_rus_m0_0["s"].sum() + df_rus_m0_0[df_rus_m0_0["dm_impl"] == 1]["dm_s"].sum()
    )
    e_o_count_0 = (
        df_rus_m0_0["o"].sum() + df_rus_m0_0[df_rus_m0_0["dm_impl"] == 1]["dm_o"].sum()
    )
    e_t_count_0 = (
        df_rus_m0_0["t"].sum() + df_rus_m0_0[df_rus_m0_0["dm_impl"] == 1]["dm_t"].sum()
    )
    e_r_count_0 = (
        df_rus_m0_0["r"].sum() + df_rus_m0_0[df_rus_m0_0["dm_impl"] == 1]["dm_r"].sum()
    )
    val_00 = (
        100
        if e_o_count_0 + e_t_count_0 == 0 or s_count_0 + e_o_count_0 + e_t_count_0 == 0
        else round(s_count_0 * 100 / (s_count_0 + e_o_count_0 + e_t_count_0), 0)
    )
    vaL_01 = (
        100
        if df_rus_m0_0["o"].sum() + df_rus_m0_0["t"].sum() == 0
        or df_rus_m0_0["s"].sum() + df_rus_m0_0["o"].sum() + df_rus_m0_0["t"].sum() == 0
        else round(
            df_rus_m0_0["s"].sum()
            * 100
            / (
                df_rus_m0_0["s"].sum() + df_rus_m0_0["o"].sum() + df_rus_m0_0["t"].sum()
            ),
            0,
        )
    )
    val_02 = (
        100
        if df_rus_m0_0["dm_o"].sum() + df_rus_m0_0["dm_t"].sum() == 0
        or df_rus_m0_0["dm_s"].sum()
        + df_rus_m0_0["dm_o"].sum()
        + df_rus_m0_0["dm_t"].sum()
        == 0
        else round(
            df_rus_m0_0["dm_s"].sum()
            * 100
            / (
                df_rus_m0_0["dm_s"].sum()
                + df_rus_m0_0["dm_o"].sum()
                + df_rus_m0_0["dm_t"].sum()
            ),
            0,
        )
    )
    # контрольные сутки
    s_count_1 = (
        df_rus_m0_1["s"].sum() + df_rus_m0_1[df_rus_m0_1["dm_impl"] == 1]["dm_s"].sum()
    )
    e_o_count_1 = (
        df_rus_m0_1["o"].sum() + df_rus_m0_1[df_rus_m0_1["dm_impl"] == 1]["dm_o"].sum()
    )
    e_t_count_1 = (
        df_rus_m0_1["t"].sum() + df_rus_m0_1[df_rus_m0_1["dm_impl"] == 1]["dm_t"].sum()
    )
    e_r_count_1 = (
        df_rus_m0_1["r"].sum() + df_rus_m0_1[df_rus_m0_1["dm_impl"] == 1]["dm_r"].sum()
    )
    val_01 = (
        100
        if e_o_count_1 + e_t_count_1 == 0 or s_count_1 + e_o_count_1 + e_t_count_1 == 0
        else round(s_count_1 * 100 / (s_count_1 + e_o_count_1 + e_t_count_1), 0)
    )
    val_11 = (
        100
        if df_rus_m0_1["o"].sum() + df_rus_m0_1["t"].sum() == 0
        or df_rus_m0_1["s"].sum() + df_rus_m0_1["o"].sum() + df_rus_m0_1["t"].sum() == 0
        else round(
            df_rus_m0_1["s"].sum()
            * 100
            / (
                df_rus_m0_1["s"].sum() + df_rus_m0_1["o"].sum() + df_rus_m0_1["t"].sum()
            ),
            0,
        )
    )
    val_12 = (
        100
        if df_rus_m0_1["dm_o"].sum() + df_rus_m0_1["dm_t"].sum() == 0
        or df_rus_m0_1["dm_s"].sum()
        + df_rus_m0_1["dm_o"].sum()
        + df_rus_m0_1["dm_t"].sum()
        == 0
        else round(
            df_rus_m0_1["dm_s"].sum()
            * 100
            / (
                df_rus_m0_1["dm_s"].sum()
                + df_rus_m0_1["dm_o"].sum()
                + df_rus_m0_1["dm_t"].sum()
            ),
            0,
        )
    )

    val_13 = s_val(s_count_0, s_count_1)
    val_14 = s_val(df_rus_m0_0["s"].sum(), df_rus_m0_1["s"].sum())
    val_15 = s_val(df_rus_m0_0["dm_s"].sum(), df_rus_m0_1["dm_s"].sum())
    val_16 = s_val(e_o_count_0, e_o_count_1)
    val_17 = s_val(df_rus_m0_0["o"].sum(), df_rus_m0_1["o"].sum())
    val_18 = s_val(df_rus_m0_0["dm_o"].sum(), df_rus_m0_1["dm_o"].sum())

    val_19 = s_val(e_t_count_0, e_t_count_1)
    val_110 = s_val(df_rus_m0_0["t"].sum(), df_rus_m0_1["t"].sum())
    val_111 = s_val(df_rus_m0_0["dm_t"].sum(), df_rus_m0_1["dm_t"].sum())
    val_112 = s_val(e_r_count_0, e_r_count_1)
    val_113 = s_val(df_rus_m0_0["r"].sum(), df_rus_m0_1["r"].sum())
    val_114 = s_val(df_rus_m0_0["dm_r"].sum(), df_rus_m0_1["dm_r"].sum())

    mes_rus_0 = f"""
    В ЦЕЛОМ ПО РФ

    Доля успешных записей (без учета федеральных ошибок и черновиков):
        - ВСЕГО:    {val_00}% (за предыдущие сутки {vaL_01}%)
        - КУ ФЭР:   {vaL_01}% (за предыдущие сутки {val_11}%)
        - Витрины:  {val_02}% (за предыдущие сутки {val_12}%)

        ▫️ Кол-во успешных записей через ЕПГУ: 
            - ВСЕГО:     {s_count_0} ({val_13} за сутки)
            - КУ ФЭР:    {df_rus_m0_0["s"].sum()} ({val_14} за сутки)
            - витрины:   {df_rus_m0_0[df_rus_m0_0["dm_impl"] == 1]["dm_s"].sum()} ({val_15} за сутки)

        ▫️ Кол-во орг ошибок субъектов РФ: 
            - ВСЕГО:     {e_o_count_0} ({val_16} за сутки)
            - КУ ФЭР:    {df_rus_m0_0["o"].sum()} ({val_17} за сутки)
            - витрины:   {df_rus_m0_0[df_rus_m0_0["dm_impl"] == 1]["dm_o"].sum()} ({val_18} за сутки)

        ▫️ Кол-во тех ошибок субъектов РФ: 
            - ВСЕГО:     {e_t_count_0} ({val_19} за сутки)
            - КУ ФЭР:    {df_rus_m0_0["t"].sum()} ({val_110} за сутки)
            - витрины:   {df_rus_m0_0[df_rus_m0_0["dm_impl"] == 1]["dm_t"].sum()} ({val_111} за сутки)

        ▫️ Кол-во ошибок федерального уровня: 
            - ВСЕГО:     {e_r_count_0} ({val_112} за сутки)
            - КУ ФЭР:    {df_rus_m0_0["r"].sum()} ({val_113} за сутки)
            - витрины:   {df_rus_m0_0["dm_r"].sum()} ({val_114} за сутки)
    """

    # сообщение 2
    rus_1_stoplist = [
        "г. Байконур",
        "Федеральное медико-биологическое агентство",
        "Донецкая Народная Республика",
        "Запорожская область",
        "Луганская Народная республика",
        "Херсонская область",
        "Харьковская область",
    ]
    df_rus_calc["s_ratio"] = df_rus_calc.apply(
        lambda row: (
            1
            if (
                (row[4] + row[5] + row[9] + row[10]) == 0
                or (row[3] + row[4] + row[5] + row[8] + row[9] + row[10]) == 0
            )
            else (row[3] + row[8])
            / (row[3] + row[4] + row[5] + row[8] + row[9] + row[10])
        ),
        axis=1,
    )
    df_rus_m1_0 = df_rus_calc[
        (df_rus_calc["date"] == max(date_list))
        & (~df_rus_calc["reg_name"].isin(rus_1_stoplist))
    ]
    df_rus_m1_1 = df_rus_calc[
        (df_rus_calc["date"] == min(date_list))
        & (~df_rus_calc["reg_name"].isin(rus_1_stoplist))
    ]

    df_comp_m1 = (
        df_rus_m1_0[["reg_name", "s_ratio"]]
        .rename(columns={"s_ratio": "s_ratio_0"})
        .merge(
            df_rus_m1_1[["reg_name", "s_ratio"]].rename(
                columns={"s_ratio": "s_ratio_1"}
            ),
            on="reg_name",
            how="inner",
        )
    )
    df_comp_m1["increase"] = df_comp_m1["s_ratio_0"] - df_comp_m1["s_ratio_1"]

    df_comp_m1 = df_comp_m1.sort_values(
        by=["s_ratio_0", "reg_name"], ascending=False
    ).reset_index(drop=True)
    list_m1_good = []
    for index, row in df_comp_m1.iterrows():
        if index < 5:
            list_m1_good += [
                row[0],
                str(round(row[1] * 100, 1)),
                (
                    "+" + str(round(row[3] * 100, 1))
                    if row[3] > 0
                    else str(round(row[3] * 100, 1))
                ),
            ]
        else:
            break

    df_comp_m1 = df_comp_m1.sort_values(
        by=["s_ratio_0", "reg_name"], ascending=True
    ).reset_index(drop=True)
    list_m1_bad = []
    for index, row in df_comp_m1.iterrows():
        if index < 5:
            list_m1_bad += [
                row[0],
                str(round(row[1] * 100, 1)),
                (
                    "+" + str(round(row[3] * 100, 1))
                    if row[3] > 0
                    else str(round(row[3] * 100, 1))
                ),
            ]
        else:
            break

    mes_rus_1 = f"""
    ЛУЧШИЕ 5 субъектов РФ по доле успешных записей (прирост за сутки):

        1. {list_m1_good[0]}: {list_m1_good[1]}% ({list_m1_good[2]}%)
        2. {list_m1_good[3]}: {list_m1_good[4]}% ({list_m1_good[5]}%)
        3. {list_m1_good[6]}: {list_m1_good[7]}% ({list_m1_good[8]}%)
        4. {list_m1_good[9]}: {list_m1_good[10]}% ({list_m1_good[11]}%)
        5. {list_m1_good[12]}: {list_m1_good[13]}% ({list_m1_good[14]}%)

    ХУДШИЕ 5 субъектов РФ по доле успешных записей (прирост за сутки):

        1. {list_m1_bad[0]}: {list_m1_bad[1]}% ({list_m1_bad[2]}%)
        2. {list_m1_bad[3]}: {list_m1_bad[4]}% ({list_m1_bad[5]}%)
        3. {list_m1_bad[6]}: {list_m1_bad[7]}% ({list_m1_bad[8]}%)
        4. {list_m1_bad[9]}: {list_m1_bad[10]}% ({list_m1_bad[11]}%)
        5. {list_m1_bad[12]}: {list_m1_bad[13]}% ({list_m1_bad[14]}%)
    """
    # сообщение 3
    df_comp_m2 = df_comp_m1.copy()
    df_comp_m2 = df_comp_m2.sort_values(
        by=["increase", "reg_name"], ascending=False
    ).reset_index(drop=True)
    list_m2_good = []
    for index, row in df_comp_m2.iterrows():
        if index < 5:
            list_m2_good += [
                row[0],
                (
                    "+" + str(round(row[3] * 100, 1))
                    if row[3] > 0
                    else str(round(row[3] * 100, 1))
                ),
                str(round(row[1] * 100, 1)),
            ]
        else:
            break

    df_comp_m2 = df_comp_m2.sort_values(
        by=["increase", "reg_name"], ascending=True
    ).reset_index(drop=True)
    list_m2_bad = []
    for index, row in df_comp_m2.iterrows():
        if index < 5:
            list_m2_bad += [
                row[0],
                (
                    "+" + str(round(row[3] * 100, 1))
                    if row[3] > 0
                    else str(round(row[3] * 100, 1))
                ),
                str(round(row[1] * 100, 1)),
            ]
        else:
            break

    mes_rus_2 = f"""
    ТОП-5 субъектов РФ, где доля успешных записей ВЫРОСЛА сильнее всего (доля успешных записей за сутки):

        1. {list_m2_good[0]}: {list_m2_good[1]}% ({list_m2_good[2]}%)
        2. {list_m2_good[3]}: {list_m2_good[4]}% ({list_m2_good[5]}%)
        3. {list_m2_good[6]}: {list_m2_good[7]}% ({list_m2_good[8]}%)
        4. {list_m2_good[9]}: {list_m2_good[10]}% ({list_m2_good[11]}%)
        5. {list_m2_good[12]}: {list_m2_good[13]}% ({list_m2_good[14]}%)

    ТОП-5 субъектов РФ, где доля успешных записей СНИЗИЛАСЬ сильнее всего (доля успешных записей за сутки):

        1. {list_m2_bad[0]}: {list_m2_bad[1]}% ({list_m2_bad[2]}%)
        2. {list_m2_bad[3]}: {list_m2_bad[4]}% ({list_m2_bad[5]}%)
        3. {list_m2_bad[6]}: {list_m2_bad[7]}% ({list_m2_bad[8]}%)
        4. {list_m2_bad[9]}: {list_m2_bad[10]}% ({list_m2_bad[11]}%)
        5. {list_m2_bad[12]}: {list_m2_bad[13]}% ({list_m2_bad[14]}%)
    """
    return mes_rus_0, mes_rus_1, mes_rus_2, pdf_path


def rating(reg_name, db_path):
    """
    Рейтинг субъектов (сегодня, вчера)
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_rating = """
    SELECT dr.reg_name, dr.date, dr.s, dr.o, dr.t, dr.dm_s, dr.dm_o, dm_t
    FROM data_rus dr
    WHERE dr.public_flag = 1;
    """
    df_rating = pd.DataFrame(
        cur.execute(query_rating).fetchall(),
        columns=["reg_name", "date", "s", "o", "t", "dm_s", "dm_o", "dm_t"],
    )

    def rating_prep(row):
        if (
            row[3] + row[4] + row[6] + row[7] == 0
            or row[2] + row[5] + row[3] + row[4] + row[6] + row[7] == 0
        ):
            return 1
        else:
            return (row[2] + row[5]) / (
                row[2] + row[5] + row[3] + row[4] + row[6] + row[7]
            )

    df_rating["ratio"] = df_rating.apply(rating_prep, axis=1)
    date_list = df_rating["date"].unique().tolist()
    df_rating_0 = (
        df_rating[df_rating["date"] == max(date_list)]
        .sort_values(by=["ratio", "s", "dm_s"], ascending=False)
        .reset_index(drop=True)
    )
    df_rating_0["rating"] = df_rating_0.apply(lambda row: row.name + 1, axis=1)
    df_rating_1 = (
        df_rating[df_rating["date"] == min(date_list)]
        .sort_values(by=["ratio", "s", "dm_s"], ascending=False)
        .reset_index(drop=True)
    )
    df_rating_1["rating"] = df_rating_1.apply(lambda row: row.name + 1, axis=1)
    rating_0 = df_rating_0[df_rating_0["reg_name"] == reg_name]["rating"].values[0]
    rating_1 = df_rating_1[df_rating_1["reg_name"] == reg_name]["rating"].values[0]
    return rating_0, rating_1


def calc_filling_sub(df_e, df_s, df_dm, db_path):
    """
    Расчёт и заливка данных для региональной рассылки
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    df_reg_ref = reg_ref(db_path)

    # корректные названия регионов
    df_e_rr = df_e.merge(
        df_reg_ref, left_on="Название субъекта РФ", right_on="reg_name_data", how="left"
    ).drop("reg_name_data", axis=1)
    df_s_rr = df_s.merge(
        df_reg_ref, left_on="Название субъекта РФ", right_on="reg_name_data", how="left"
    ).drop("reg_name_data", axis=1)
    df_dm_rr = df_dm.merge(
        df_reg_ref, left_on="Регион", right_on="reg_name_data", how="left"
    ).drop("reg_name_data", axis=1)

    # дата данных
    date_data = df_e_rr.loc[0, "create_ts"].strftime("%Y-%m-%d")

    # ось для резутьтатов - ПРАВИЛЬНАЯ, ИЗ БД
    query = """
    SELECT DISTINCT r.reg_name_rus, r.public_flag FROM regions r
    """
    df_axis = pd.DataFrame(
        cur.execute(query).fetchall(), columns=["reg_name", "public_flag"]
    ).drop_duplicates()

    # добавление столбца сортировки по МО
    df_e_rr["mo_group"] = df_e_rr.apply(
        lambda row: "_".join([str(row[15]), str(row[6]), str(row[7])]), axis=1
    )
    df_s_rr["mo_group"] = df_s_rr.apply(
        lambda row: "_".join([str(row[9]), str(row[3]), str(row[4])]), axis=1
    )

    df_e_mog = (
        df_e_rr.groupby(
            ["mo_group", "reg_name", "mo_oid", "mo_name", "final_state", "error_type"]
        )["Тип записи"]
        .count()
        .reset_index()
        .rename(columns={"Тип записи": "error_count"})
    )
    df_e_mog.insert(1, "date", date_data)
    df_s_mog = (
        df_s_rr.groupby(["mo_group", "reg_name", "sp_oid", "sp_name"])["Тип записи"]
        .count()
        .reset_index()
        .rename(columns={"Тип записи": "s_count"})
    )
    df_s_mog.insert(1, "date", date_data)
    # укорачивание чересчур длинных названий МО
    df_s_mog["mo_cut_name"] = df_s_mog["sp_name"].apply(
        lambda x: (x[:128] + "...") if len(x) > 128 else x
    )
    df_e_mog["mo_cut_name"] = df_e_mog["mo_name"].apply(
        lambda x: (x[:128] + "...") if len(x) > 128 else x
    )

    df_e_mog_sh = df_e_mog[
        [
            "mo_group",
            "reg_name",
            "date",
            "mo_cut_name",
            "final_state",
            "error_type",
            "error_count",
        ]
    ]
    df_s_mog_sh = df_s_mog[["mo_group", "reg_name", "date", "mo_cut_name", "s_count"]]
    df_mog_sh = df_s_mog_sh.merge(
        df_e_mog_sh, on=["mo_group", "date", "reg_name"], how="outer"
    ).rename(
        columns={"mo_cut_name_x": "mo_cut_name_s", "mo_cut_name_y": "mo_cut_name_e"}
    )
    df_mog_sh = df_axis.merge(df_mog_sh, on="reg_name", how="left")
    df_mog_sh["s_count"] = df_mog_sh["s_count"].fillna(0)
    df_mog_sh["error_count"] = df_mog_sh["error_count"].fillna(0)
    df_mog_sh["date"] = df_mog_sh["date"].fillna(date_data)
    df_mog_sh = df_mog_sh.astype({"s_count": int, "error_count": int})

    # для сообщений 3 и 4, где учитываются только организационные ошибки и успешные записи
    df_mog_sh_group_oe = (
        df_mog_sh[df_mog_sh["error_type"] == "o"]
        .groupby("mo_group")["error_count"]
        .sum()
        .reset_index()
    )
    df_mog_sh_group_s = df_mog_sh.groupby("mo_group")["s_count"].sum().reset_index()
    df_mog_sh_group = df_mog_sh_group_s.merge(
        df_mog_sh_group_oe, on="mo_group", how="outer"
    ).fillna(0)
    df_mog_sh_db = (
        df_mog_sh[["reg_name", "mo_group", "date", "mo_cut_name_s", "mo_cut_name_e"]]
        .merge(df_mog_sh_group, on="mo_group", how="left")
        .drop_duplicates()
    )
    df_mog_sh_db["s_count"] = df_mog_sh_db["s_count"].fillna(0)
    df_mog_sh_db["error_count"] = df_mog_sh_db["error_count"].fillna(0)
    # + доля успешных записей по МО с учётом только организационных ошибок
    df_e = (
        df_e_rr[df_e_rr["error_type"] == "o"]
        .groupby("mo_group")["Тип записи"]
        .count()
        .reset_index()
        .rename(columns={"Тип записи": "error_count"})
    )
    df_s = (
        df_s_rr.groupby("mo_group")["Тип записи"]
        .count()
        .reset_index()
        .rename(columns={"Тип записи": "s_count"})
    )
    df_ratio = df_s.merge(df_e, on="mo_group", how="outer").fillna(0)
    df_ratio["s_ratio"] = df_ratio.apply(
        lambda row: (
            1 if (row[2] == 0) or (row[1] + row[2] == 0) else row[1] / (row[1] + row[2])
        ),
        axis=1,
    )
    df_mog_sh_db = df_mog_sh_db.merge(
        df_ratio[["mo_group", "s_ratio"]], on="mo_group", how="left"
    )
    df_mog_sh_db["s_ratio"] = df_mog_sh_db["s_ratio"].fillna(1)
    df_mog_sh_db = df_mog_sh_db.astype({"s_count": int, "error_count": int})
    df_mog_sh_db["date"] = pd.to_datetime(df_mog_sh_db["date"], format="mixed")
    # df_mog_sh_db - заливаем в БД
    query = """
    SELECT DISTINCT date(dm.date) FROM data_mo dm;
    """
    date_db = cur.execute(query).fetchall()
    date_db_list = [x[0] for x in date_db]
    if date_data in date_db_list:
        result_0 = f"❓Расчёт данных МО для региональной рассылки выполнен, но данные с датой {date_data} уже есть в БД."
    else:
        df_mog_sh_db.to_sql("data_mo", conn, if_exists="append", index=False)
        # удаление данных старше, чем текущение - 1 день
        min_db_date = df_mog_sh_db.loc[0, "date"] - timedelta(days=1)
        min_db_date_query = min_db_date.strftime("%Y-%m-%d")
        query_del_date = """
        DELETE FROM data_mo WHERE date < ?
        """
        cur.execute(query_del_date, (min_db_date_query,))
        conn.commit()
        result_0 = f"👍Расчёт данных МО для региональной рассылки выполнен, данные на {date_data} залиты в БД"

    # типы ошибок, группировка - finas_state и регион, засим всё это поместить в БД
    df_final_state = (
        df_e_rr.groupby(["reg_name", "final_state", "error_type"])["Тип записи"]
        .count()
        .reset_index()
        .rename(columns={"Тип записи": "error_count"})
    )
    df_final_state = df_axis.merge(df_final_state, on="reg_name", how="left")
    df_final_state.drop("public_flag", axis=1, inplace=True)

    df_final_state.insert(1, "date", date_data)
    df_final_state = df_final_state.merge(
        df_s_rr.groupby("reg_name")["Тип записи"]
        .count()
        .reset_index()
        .rename(columns={"Тип записи": "s_count"}),
        on="reg_name",
        how="outer",
    )
    df_final_state["s_count"] = df_final_state["s_count"].fillna(0)
    df_final_state["s_count"] = df_final_state["s_count"].astype(int)
    df_final_state["date"] = pd.to_datetime(df_final_state["date"], format="mixed")
    df_final_state["error_count"] = df_final_state["error_count"].fillna(0)
    query = """
    SELECT DISTINCT date(ds.date) FROM data_sub ds;
    """
    date_db = cur.execute(query).fetchall()
    date_db_list = [x[0] for x in date_db]
    if date_data in date_db_list:
        result_1 = f"❓Расчёт данных субъектов РФ для региональной рассылки выполнен, но данные с датой {date_data} уже есть в БД."
    else:
        df_final_state.to_sql("data_sub", conn, if_exists="append", index=False)
        # удаление данных старше, чем текущение - 1 день
        min_db_date = df_final_state.loc[0, "date"] - timedelta(days=1)
        min_db_date_query = min_db_date.strftime("%Y-%m-%d")
        query_del_date = """
        DELETE FROM data_sub WHERE date < ?
        """
        cur.execute(query_del_date, (min_db_date_query,))
        conn.commit()
        result_1 = f"👍Расчёт данных субъектов РФ для региональной рассылки выполнен, данные на {date_data} залиты в БД"
    return result_0, result_1


def sub_messages(dir_path, db_path, reg_name):
    """
    Формирование региональной рассылки (сообщения и файлы excel + pdf)
    """
    global df_error_t_01 # ТУТ!!!
    sub_excel_path = "/".join([dir_path, "files/sub_report.xlsx"])
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # файлы
    query_mo = """
    SELECT *
    FROM data_mo dr;
    """
    df_mo = pd.DataFrame(
        cur.execute(query_mo).fetchall(),
        columns=[
            "reg_name",
            "mo_group",
            "date",
            "mo_cut_name_s",
            "mo_cut_name_e",
            "s_count",
            "error_count",
            "s_ratio",
        ],
    )
    df_mo["date"] = pd.to_datetime(df_mo["date"], format="%Y-%m-%d %H:%M:%S")
    df_mo_reg = df_mo[df_mo["reg_name"] == reg_name]
    df_axis = df_mo_reg[["reg_name", "mo_group"]].drop_duplicates()
    df_mo_reg["mo_cut_name_s"] = df_mo_reg["mo_cut_name_s"].fillna("я")
    df_mo_reg["mo_cut_name_e"] = df_mo_reg["mo_cut_name_e"].fillna("я")
    date_list = df_mo_reg["date"].unique().tolist()
    df_mo_0 = df_mo_reg[df_mo_reg["date"] == max(date_list)]
    df_mo_1 = df_mo_reg[df_mo_reg["date"] == min(date_list)]
    # если в этих df ничего нет?
    try:

        def mo_status(df):
            df.insert(3, "oid_sp", df["mo_group"].apply(lambda x: x.split("_")[1]))
            df.insert(4, "name_sp", df.apply(lambda row: min(row[4], row[5]), axis=1))
            df = df[
                ["mo_group", "oid_sp", "name_sp", "error_count", "s_count", "s_ratio"]
            ].reset_index(drop=True)
            return df

        df_mo_0 = mo_status(df_mo_0)
        df_mo_1 = mo_status(df_mo_1)

        df_mo_a0 = df_axis.merge(df_mo_0, on="mo_group", how="left")
        df_mo_a0["error_count"] = df_mo_a0["error_count"].fillna(0)
        df_mo_a0["s_count"] = df_mo_a0["s_count"].fillna(0)
        df_mo_a0["s_ratio"] = df_mo_a0["s_ratio"].fillna(1)
        df_mo_a1 = df_axis.merge(df_mo_1, on="mo_group", how="left")
        df_mo_a1["error_count"] = df_mo_a1["error_count"].fillna(0)
        df_mo_a1["s_count"] = df_mo_a1["s_count"].fillna(0)
        df_mo_a1["s_ratio"] = df_mo_a1["s_ratio"].fillna(1)
        df_mo = df_mo_a1.merge(df_mo_a0, on="mo_group", how="outer").fillna("я")
        df_mo.insert(2, "oid", df_mo.apply(lambda row: min(row[2], row[8]), axis=1))
        df_mo.insert(
            3, "sp_name", df_mo.apply(lambda row: min(row[4], row[10]), axis=1)
        )
        df_mo = df_mo[
            [
                "oid",
                "sp_name",
                "error_count_x",
                "s_count_x",
                "s_ratio_x",
                "error_count_y",
                "s_count_y",
                "s_ratio_y",
            ]
        ]
        df_mo = df_mo.sort_values(by="s_ratio_y", ascending=False).reset_index(
            drop=True
        )
        df_mo_excel = df_mo.copy()
        df_mo_excel = df_mo_excel.rename(
            columns={
                "sp_name": "Наименование СП",
                "error_count_x": "Кол-во орг. ошибок",
                "s_count_x": "Кол-во успешных записей",
                "s_ratio_x": "Доля успешных записей",
                "error_count_y": "Кол-во орг. ошибок",
                "s_count_y": "Кол-во успешных записей",
                "s_ratio_y": "Доля успешных записей",
            }
        )
    except Exception:
        df_mo_excel = pd.DataFrame(
            columns=[
                "oid",
                "Наименование СП",
                "Кол-во орг. ошибок",
                "Кол-во успешных записей",
                "Доля успешных записей",
                "Кол-во орг. ошибок",
                "Кол-во успешных записей",
                "Доля успешных записей",
            ]
        )

    # перенос данных в excel
    wb = xw.Book(sub_excel_path)
    app = wb.app
    # выбор листа для вставки данных
    sht = wb.sheets["sub"]
    # очистка диапазона
    sht.range("A5:O999999").clear_contents()
    # вставка данных из DataFrame в ячейку
    sht.range("A2").value = (
        "Рейтинг структурных подразделений МО (данные витрин ЕПГУ не уситываются)"
    )
    sht.range("A3").value = reg_name
    sht.range("D4").value = min(date_list)
    sht.range("G4").value = max(date_list)
    sht.range("A5").value = df_mo_excel
    sht.range("A5").value = "№№"
    # сохранение изменений
    wb.save()
    sub_save_excel_path = "".join(
        [
            dir_path + "/",
            "/files/",
            "sub_report_",
            max(date_list).strftime("%Y-%m-%d"),
            ".xlsx",
        ]
    )
    wb.save(sub_save_excel_path)
    sht_pdf = wb.sheets["sub"]
    sht_pdf.page_setup.print_area = "".join(["$A$1:$I$", str(df_mo_excel.shape[0] + 5)])
    pdf_path = "".join(
        [
            dir_path + "/",
            "/files/",
            "sub_report_",
            max(date_list).strftime("%Y-%m-%d"),
            ".pdf",
        ]
    )
    sht_pdf.to_pdf(path=pdf_path, show=False)
    # сохранение изменений и окончание этого вот всего
    wb.save()
    wb.close()
    app.kill()

    # сообщение 1 (а по сути 2)

    query_ms0 = """
    SELECT *
    FROM data_rus dr
    WHERE dr.reg_name = ?
    ORDER BY dr.date DESC
    """
    ms0 = cur.execute(query_ms0, (reg_name,)).fetchall()

    val_00 = (
        100.0
        if (ms0[0][3] + ms0[0][8] + ms0[0][4] + ms0[0][9]) == 0
        or ms0[0][4] + ms0[0][9] == 0
        else round(
            (ms0[0][3] + ms0[0][8])
            * 100
            / (ms0[0][3] + ms0[0][8] + ms0[0][4] + ms0[0][9]),
            1,
        )
    )
    val_01 = (
        100.0
        if (ms0[1][3] + ms0[1][8] + ms0[1][4] + ms0[1][9]) == 0
        or ms0[1][4] + ms0[1][9] == 0
        else round(
            (ms0[1][3] + ms0[1][8])
            * 100
            / (ms0[1][3] + ms0[1][8] + ms0[1][4] + ms0[1][9]),
            1,
        )
    )
    val_10 = (
        100.0
        if (ms0[0][3] + ms0[0][4]) == 0 or ms0[0][4] == 0
        else round(ms0[0][3] * 100 / (ms0[0][3] + ms0[0][4]), 1)
    )
    val_11 = (
        100.0
        if (ms0[1][3] + ms0[1][4]) == 0 or ms0[1][4] == 0
        else round(ms0[1][3] * 100 / (ms0[1][3] + ms0[1][4]), 1)
    )
    if ms0[0][7] == 1 and ms0[1][7] == 1:
        v_0 = (
            100.0
            if (ms0[0][8] + ms0[0][9]) == 0 or ms0[0][9] == 0
            else round(ms0[0][9] * 100 / (ms0[0][8] + ms0[0][9]), 1)
        )
        v_1 = (
            100.0
            if (ms0[1][8] + ms0[1][9]) == 0 or ms0[1][9] == 0
            else round(ms0[1][9] * 100 / (ms0[1][8] + ms0[1][9]), 1)
        )
        val_v = f"""
        - витрины:  {v_0}% (за предыдущие сутки – {v_1}%)"""
    elif ms0[0][7] == 1 and ms0[1][7] == 0:
        v_0 = (
            100.0
            if (ms0[0][8] + ms0[0][9]) == 0 or ms0[0][9] == 0
            else round(ms0[0][9] * 100 / (ms0[0][8] + ms0[0][9]), 1)
        )
        # v_1 = 100.0 if (ms0[1][8]+ms0[1][9])==0 or ms0[1][9]==0 else round(ms0[1][9]*100/(ms0[1][8]+ms0[1][9]), 1)
        val_v = f"""
        - витрины:  {v_0}% (за предыдущие сутки – нет)"""
    elif ms0[0][7] == 0 and ms0[1][7] == 1:
        # v_0 = 100.0 if (ms0[0][8]+ms0[0][9])==0 or ms0[0][9]==0 else round(ms0[0][9]*100/(ms0[0][8]+ms0[0][9]), 1)
        v_1 = (
            100.0
            if (ms0[1][8] + ms0[1][9]) == 0 or ms0[1][9] == 0
            else round(ms0[1][9] * 100 / (ms0[1][8] + ms0[1][9]), 1)
        )
        val_v = f"""
        - витрины:  нет (за предыдущие сутки – {v_1}%)"""
    else:
        val_v = """
        - витрины:  нет
    """
    rating_0, rating_1 = rating(reg_name, db_path)
    query_rating_c = """
    SELECT dr.date, count(*)
    FROM data_rus dr
    WHERE dr.public_flag = 1
    GROUP BY dr.date
    ORDER BY dr.date DESC;
    """
    r_c = cur.execute(query_rating_c).fetchall()

    def s_val(v_0, v_1):
        if v_0 > v_1:
            return "+" + str(round(v_0 - v_1, 1))
        elif v_0 < v_1:
            return "-" + str(round(v_1 - v_0, 1))
        else:
            return "без изменений"

    if ms0[0][7] == 1 and ms0[1][7] == 1:
        v_s = f"""- витрины:  {ms0[0][8]} ({s_val(ms0[0][8], ms0[1][8])} за сутки)
    """
        v_o = f"""- витрины:  {ms0[0][9]} ({s_val(ms0[0][9], ms0[1][9])} за сутки)
    """
        v_t = f"""- витрины:  {ms0[0][10]} ({s_val(ms0[0][10], ms0[1][10])} за сутки)
    """
    elif ms0[0][7] == 1 and ms0[1][7] == 0:
        v_s = f"""- витрины:  {ms0[0][8]} (предыдущие сутки - нет)
    """
        v_o = f"""- витрины:  {ms0[0][9]} (предыдущие сутки - нет)
    """
        v_t = f"""- витрины:  {ms0[0][10]} (предыдущие сутки - нет)
    """
    elif ms0[0][7] == 0 and ms0[1][7] == 1:
        v_s = f"""- витрины:  нет ({s_val(0, ms0[1][8])} за сутки)
    """
        v_o = f"""- витрины:  нет ({s_val(0, ms0[1][9])} за сутки)
    """
        v_t = f"""- витрины:  нет ({s_val(0, ms0[1][10])} за сутки)
    """
    else:
        v_s = """- витрины:   нет
    """
        v_o = """- витрины:   нет
    """
        v_t = """- витрины:   нет
    """

    mes_sub_0 = f"""
    {reg_name}

    Доля успешных записей (без учета ошибок федерального уровня и черновиков):
        - ВСЕГО:    {val_00}% (за предыдущие сутки {val_10}%)
        - КУ ФЭР:   {val_01}% (за предыдущие сутки {val_11}%){val_v}
    {rating_0} место в рейтинге субъектов РФ (1 – лучший; {r_c[0][1]} – худший)
    предыдущие сутки - {rating_1} место в рейтинге

        ▫️ Кол-во успешных записей через ЕПГУ:
            - ВСЕГО:     {ms0[0][3]+ms0[0][8]} ({s_val(ms0[0][3]+ms0[0][8], ms0[1][3]+ms0[1][8])} за сутки)
            - КУ ФЭР:    {ms0[0][3]} ({s_val(ms0[0][3], ms0[1][3])} за сутки)
            {v_s}
        ▫️ Кол-во орг. ошибок субъекта РФ:
            - ВСЕГО:     {ms0[0][4]+ms0[0][9]} ({s_val(ms0[0][4]+ms0[0][9], ms0[1][4]+ms0[1][9])} за сутки)
            - КУ ФЭР:    {ms0[0][4]} ({s_val(ms0[0][4], ms0[1][4])} за сутки)
            {v_o}
        ▫️ Кол-во тех. ошибок субъекта РФ:
            - ВСЕГО:     {ms0[0][5]+ms0[0][10]} ({s_val(ms0[0][5]+ms0[0][10], ms0[1][5]+ms0[1][10])} за сутки)
            - КУ ФЭР:    {ms0[0][5]} ({s_val(ms0[0][5], ms0[1][5])} за сутки)
            {v_t}
    #статистика_запись
    """

    # сообщение 2 (а по сути 3)
    try:

        def kw(s0, r0, e0, s1, r1, e1):
            """`чем kw меньше, тем МО хуже"""
            kw = math.exp(s0 + r0 * 100 - e0) / math.exp(s1 + r1 * 100 - e1)
            # kw = math.exp(s0+r0*100)/math.exp(s1+r1*100)
            return kw

        df_mo["kw"] = df_mo.apply(
            lambda row: kw(row[6], row[7], row[5], row[3], row[4], row[2]), axis=1
        )
        df_mo_n = df_mo.copy()
        df_mo = df_mo.sort_values(by="kw").head(5)
        mes_sub_1_dict = []
        for index, row in df_mo.iterrows():
            mes_sub_1_dict += [
                row[1],
                round(row[7] * 100, 1),
                (
                    "+" + str(round((row[7] - row[4]) * 100, 1))
                    if row[7] > row[4]
                    else str(round((row[7] - row[4]) * 100, 1))
                ),
            ]
        mes_sub_1_str = ""
        k = 0
        for i in range(1, 6):
            try:
                mes_sub_1_str += (
                    str(i)
                    + ". "
                    + str(mes_sub_1_dict[k])
                    + ": "
                    + str(mes_sub_1_dict[k + 1])
                    + "% ("
                    + str(mes_sub_1_dict[k + 2])
                    + "%)\n"
                )
                k += 3
            except IndexError:
                pass

        mes_sub_1 = (
            f"""
    Медицинские организации с самой низкой долей успешных записей и худшей динамикой (прирост за сутки)
    в рейтинге МО учитываются только организационные ошибки и успешные записи: 
    """
            + mes_sub_1_str
            + "#доступность_записи"
        )
    except Exception:
        mes_sub_1 = f"""
    Медицинские организации с самой низкой долей успешных записей и худшей динамикой (прирост за сутки)
    в рейтинге МО учитываются только организационные ошибки и успешные записи:
    #доступность_записи
    """
        df_mo_n = pd.DataFrame()

    # сообщение 3 (а по сути 4)
    try:
        df_mo_n["akw"] = 1 / df_mo_n["kw"]
        df_mo_n = df_mo_n.sort_values(
            by=["error_count_y", "akw"], ascending=False
        ).head(5)
        df_mo_n["error_count_x"] = df_mo_n["error_count_x"].astype(int)
        df_mo_n["error_count_y"] = df_mo_n["error_count_y"].astype(int)
        mes_sub_2_dict = []
        for index, row in df_mo_n.iterrows():
            mes_sub_2_dict += [
                row[1],
                row[5],
                row[2],
                round(row[7] * 100, 1),
                round(row[4] * 100, 1),
            ]
        mes_sub_2_str = ""
        n = 0
        for i in range(1, 6):
            try:
                mes_sub_2_str += (
                    str(i)
                    + ". "
                    + str(mes_sub_2_dict[n])
                    + ": "
                    + str(mes_sub_2_dict[n + 1])
                    + " орг. ош.; за предыдущие сутки "
                    + str(mes_sub_2_dict[n + 2])
                    + " ("
                    + str(mes_sub_2_dict[n + 3])
                    + "%; за предыдущие сутки "
                    + str(mes_sub_2_dict[n + 4])
                    + "%)\n"
                )
                n += 5
            except IndexError:
                pass

        mes_sub_2 = (
            f"""
    Медицинские организации с самым большим количеством орг. ошибок и худшей динамикой (в скобках процент успешных записей)
    эти МО сильнее всего повлияли на рейтинг:
    """
            + mes_sub_2_str
            + "#орг_ошибки"
        )
    except Exception:
        mes_sub_2 = f"""
    Медицинские организации с самым большим количеством орг. ошибок и худшей динамикой (в скобках процент успешных записей)
    эти МО сильнее всего повлияли на рейтинг:
    #орг_ошибки
    """

    # сообщение 4 (а по сути 5)

    query_axis_t = """
    SELECT DISTINCT et.final_state
    FROM error_type et
    WHERE et.error_type = 't';
    """

    df_axis_t = pd.DataFrame(
        cur.execute(query_axis_t).fetchall(),
        columns=["fs"],
    )

    query_ms4 = """
    SELECT *
    FROM data_sub ds
    WHERE ds.reg_name = ?
    """
    ms4 = cur.execute(query_ms4, (reg_name,)).fetchall()

    df_ms4 = pd.DataFrame(
        ms4,
        columns=[
            "reg_name",
            "date",
            "final_state",
            "error_type",
            "error_count",
            "s_count",
        ],
    )
    df_ms4["date"] = pd.to_datetime(df_ms4["date"], format="%Y-%m-%d %H:%M:%S")
    date_list = df_ms4["date"].unique().tolist()
    df_ms4_0 = df_ms4[df_ms4["date"] == max(date_list)]
    df_ms4_1 = df_ms4[df_ms4["date"] == min(date_list)]
    df_ms4_0 = df_axis_t.merge(
        df_ms4_0, left_on="fs", right_on="final_state", how="left"
    )
    df_ms4_0["error_count"] = df_ms4_0["error_count"].fillna(0)
    df_ms4_0["s_count"] = df_ms4_0["s_count"].fillna(method="ffill")
    df_ms4_0["date"] = df_ms4_0["date"].fillna(method="ffill")
    df_ms4_0["s_count"] = df_ms4_0["s_count"].fillna(method="bfill")
    df_ms4_0["date"] = df_ms4_0["date"].fillna(method="bfill")
    df_ms4_0 = df_ms4_0[["fs", "error_count", "s_count"]]
    df_ms4_1 = df_axis_t.merge(
        df_ms4_1, left_on="fs", right_on="final_state", how="left"
    )
    df_ms4_1["error_count"] = df_ms4_1["error_count"].fillna(0)
    df_ms4_1["s_count"] = df_ms4_1["s_count"].fillna(method="ffill")
    df_ms4_1["date"] = df_ms4_1["date"].fillna(method="ffill")
    df_ms4_1["s_count"] = df_ms4_1["s_count"].fillna(method="bfill")
    df_ms4_1["date"] = df_ms4_1["date"].fillna(method="bfill")
    df_ms4_1 = df_ms4_1[["fs", "error_count", "s_count"]]
    ns_0 = ms0[0][3]
    nsdm_0 = ms0[0][8]
    nt_0 = df_ms4_0["error_count"].sum()
    ntdm_0 = ms0[0][10]
    no_0 = ms0[0][4]
    nodm_0 = ms0[0][9]
    nr_0 = ms0[0][6]
    nrdm_0 = ms0[0][11]

    ns_1 = ms0[1][3]
    nsdm_1 = ms0[1][8]
    nt_1 = df_ms4_1["error_count"].sum()
    ntdm_1 = ms0[1][10]
    no_1 = ms0[1][4]
    nodm_1 = ms0[1][9]
    nr_1 = ms0[1][6]
    nrdm_1 = ms0[1][11]

    all_0 = ns_0 + nsdm_0 + nt_0 + ntdm_0 + no_0 + nodm_0 + nr_0 + nrdm_0
    all_1 = ns_1 + nsdm_1 + nt_1 + ntdm_1 + no_1 + nodm_1 + nr_1 + nrdm_1
    ratio_t_all_0 = (
        0
        if ns_0 + nsdm_0 + nt_0 + ntdm_0 + no_0 + nodm_0 + nr_0 + nrdm_0 == 0
        else round(
            (nt_0 + ntdm_0)
            * 100
            / (ns_0 + nsdm_0 + nt_0 + ntdm_0 + no_0 + nodm_0 + nr_0 + nrdm_0),
            1,
        )
    )
    ratio_t_all_1 = (
        0
        if ns_1 + nsdm_1 + nt_1 + ntdm_1 + no_1 + nodm_1 + nr_1 + nrdm_1 == 0
        else round(
            (nt_1 + ntdm_1)
            * 100
            / (ns_1 + nsdm_1 + nt_1 + ntdm_1 + no_1 + nodm_1 + nr_1 + nrdm_1),
            1,
        )
    )
    ratio_t_all_report = s_val(ratio_t_all_0, ratio_t_all_1)
    ratio_t_0 = (
        0
        if ns_0 + nt_0 + no_0 + nr_0 == 0
        else round((nt_0) * 100 / (ns_0 + nt_0 + no_0 + nr_0), 1)
    )
    ratio_t_1 = (
        0
        if ns_1 + nt_1 + no_1 + nr_1 == 0
        else round((nt_1) * 100 / (ns_1 + nt_1 + no_1 + nr_1), 1)
    )
    ratio_t_report = s_val(ratio_t_0, ratio_t_1)
    ratio_t_dm_0 = (
        0
        if nsdm_0 + ntdm_0 + nodm_0 + nrdm_0 == 0
        else round((ntdm_0) * 100 / (nsdm_0 + ntdm_0 + nodm_0 + nrdm_0), 1)
    )
    ratio_t_dm_1 = (
        0
        if nsdm_1 + ntdm_1 + nodm_1 + nrdm_1 == 0
        else round((ntdm_1) * 100 / (nsdm_1 + ntdm_1 + nodm_1 + nrdm_1), 1)
    )
    ratio_t_dm_report = s_val(ratio_t_dm_0, ratio_t_dm_1)

    # s_val(v_0, v_1)

    df_error_t_01 = df_ms4_1.merge(df_ms4_0, on="fs", how="outer")
    df_error_t_01 = df_error_t_01[["fs", "error_count_x", "error_count_y"]].rename(
        columns={"error_count_x": "error_count_1", "error_count_y": "error_count_0"}
    )
    df_error_t_01 = df_error_t_01[
        (df_error_t_01["error_count_1"] != 0) | (df_error_t_01["error_count_0"] != 0)
    ]
    df_error_t_01["rt_1"] = df_error_t_01["error_count_1"] / all_1
    df_error_t_01["rt_0"] = df_error_t_01["error_count_0"] / all_0

    mes_sub_3_part0 = f"""
    Анализ технических ошибок

    Кол-во и доля тех. ошибок субъекта РФ в общем числе ошибок и успешных записей:
        - ВСЕГО: {int(nt_0+ntdm_0)} ({s_val(int(nt_0+ntdm_0), int(nt_1+ntdm_1))} за сутки); {ratio_t_all_0}% ({ratio_t_all_report} п.п. за сутки)
        - КУ ФЭР: {int(nt_0)} ({s_val(int(nt_0), int(nt_1))} за сутки); {ratio_t_0}% ({ratio_t_report} п.п. за сутки)
        - витрины: {int(ntdm_0)} ({s_val(int(ntdm_0), int(ntdm_1))} за сутки); {ratio_t_dm_0}% ({ratio_t_dm_report} п.п. за сутки)"""

    mes_sub_3_part1 = """
    Кол-во и доля категорий тех. ошибок в общем числе ошибок и успешных записей:"""
    for index, row in df_error_t_01.iterrows():
        mes_sub_3_part1 += f"""
        ▫️ {row[0]}: {int(row[2])} ({round(row[4]*100, 1)}% из {ratio_t_0}%; за предыдущие сутки {round(row[3]*100, 1)}% из {ratio_t_1}%)"""

    dm_str = f"""
        ▫️ Ошибки витрины: {int(ntdm_0)} ({ratio_t_dm_0}% из {ratio_t_0}%; за предыдущие сутки {ratio_t_dm_1}% из {ratio_t_1}%)
    """

    mes_sub_3 = mes_sub_3_part0 + mes_sub_3_part1 + dm_str + "#тех_ошибки"

    # chat_id
    id_send_query = """
    SELECT rsd.chat_id
    FROM reg_send_data rsd
    WHERE rsd.reg_name = ?
    """
    id_send = cur.execute(id_send_query, (reg_name,)).fetchall()[0][0]

    return (
        id_send,
        mes_sub_0,
        mes_sub_1,
        mes_sub_2,
        mes_sub_3,
        sub_save_excel_path,
        pdf_path,
    )


def rus_send(id_chat, mes_rus_0, mes_rus_1, mes_rus_2, pdf_path):
    """Отправка федеральных сообщений"""
    global bot, img_path
    i0 = open("".join([img_path, "rus_2.png"]), "rb")
    i1 = open("".join([img_path, "rus_1.png"]), "rb")
    i2 = open("".join([img_path, "rus_0.png"]), "rb")
    bot.send_photo(
        id_chat,
        photo=i0,
        caption=mes_rus_0,
    )
    bot.send_photo(
        id_chat,
        photo=i1,
        caption=mes_rus_1,
    )
    bot.send_photo(
        id_chat,
        photo=i2,
        caption=mes_rus_2,
    )

    bot.send_document(
        id_chat,
        document=open(pdf_path, "rb"),
    )
    i0.close()
    i1.close()
    i2.close()


def sub_send(
    id_chat,
    mes_sub_0,
    mes_sub_1,
    mes_sub_2,
    mes_sub_3,
    sub_save_excel_path,
    pdf_path,
    date_data,
    reg_name,
):
    """Отправка региональных сообщений"""
    global bot, img_path
    i3 = open("".join([img_path, "sub_0.png"]), "rb")
    i4 = open("".join([img_path, "sub_1.png"]), "rb")
    i5 = open("".join([img_path, "sub_2.png"]), "rb")
    i6 = open("".join([img_path, "sub_3.png"]), "rb")
    i7 = open("".join([img_path, "sub_4.png"]), "rb")
    bot.send_photo(
        id_chat,
        photo=i3,
        caption=f"""
    СТАТИСТИКА ПО ЗАПИСИ К ВРАЧУ ЗА {date_data}, {reg_name}
    (без учета данных по углубленной диспансеризации и вакцинации от COVID-19)
    """,
    )
    bot.send_photo(
        id_chat,
        photo=i4,
        caption=mes_sub_0,
    )
    bot.send_photo(
        id_chat,
        photo=i5,
        caption=mes_sub_1,
    )
    bot.send_photo(
        id_chat,
        photo=i6,
        caption=mes_sub_2,
    )
    bot.send_photo(
        id_chat,
        photo=i7,
        caption=mes_sub_3,
    )

    i3.close()
    i4.close()
    i5.close()
    i6.close()
    i7.close()
    xlsx_path_sub = sub_save_excel_path
    bot.send_document(
        id_chat,
        document=open(xlsx_path_sub, "rb"),
    )
    pdf_path_sub = pdf_path
    bot.send_document(
        id_chat,
        document=open(pdf_path_sub, "rb"),
        caption="#рейтинг_СП",
    )

In [24]:
df_error_t_01

,fs,error_count_1,error_count_0,rt_1,rt_0
3,Внутренняя ошибка РМИС при создании записи,2.0,2.0,0.003578,0.000830
4,Ошибка «РМИС не ответил»,1.0,855.0,0.001789,0.354772
5,Время уже занято другим пациентом,1.0,0.0,0.001789,0.000000
6,Ошибка валидации данных пациента,24.0,52.0,0.042934,0.021577


In [8]:
# заливка картинок-шаблонов в db
# НЕ НАДО ДЕЛАТЬ, ИБО УЖЕ
img_source_path = "C:/work/new_bot/img/img_source/"
img_filling(db_path, img_source_path)

In [9]:
# делание пикчей
img_path
db_path
imd_date = "2024-10-07"

img_make(img_path, db_path, imd_date)

In [12]:
# загрузка данных и заливка в федеральную таблицу
# ТУТ ПРОСТО ТАК, СУКА, НЕ НАЖИМАТЬ
df_e, df_s, df_dm, status, fail = datastatus(e_path, s_path, dm_path, dm_etalon_path, db_path)
result = calc_filling_rus(df_e, df_s, df_dm, db_path)
print(result)

('❓Расчёт федеральной рассылки выполнен, но данные с датой 2024-10-06 уже есть в БД.', '2024-10-06')


In [4]:
# получение рейтинга субъектов
# reg_name, db_path
reg_name = "Ивановская область"
rating(reg_name, db_path)

(48, 60)

In [5]:
calc_filling_sub(df_e, df_s, df_dm, db_path)

('❓Расчёт данных МО для региональной рассылки выполнен, но данные с датой 2024-10-06 уже есть в БД.',
 '❓Расчёт данных субъектов РФ для региональной рассылки выполнен, но данные с датой 2024-10-06 уже есть в БД.')

***

## А вот и **бот**
![alt text](big_bot-1-1.png)

In [23]:
# бот

TOKEN = "7885371036:AAFtfjid_dRHh2ODPUJ-th-0h8vaxTXbtX0"  # token BigBotForAll@Bot

messages = {}
user_authorization = {}
user_status = {}
dhb_s = {}
fail = {}
user_db_edit = {}
# accept = {}
# password_flag = {}
# password_flag_add = {}
# act = {}
# act_reg = {}
# act_password = {}
# region_edit = {}

db_password = "999Abakan999"

bot = telebot.TeleBot(TOKEN)


@bot.message_handler(commands=["start"])
def start(message):
    global user_id, user_authorization, user_status, fail, dhb_s, df_e, df_s, df_dm
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    user_id = message.from_user.id
    user_authorization[user_id] = False
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query_user_status = """
    SELECT *
    FROM user_status us
    WHERE us.user_id = ?
    ORDER BY us.date_time DESC
    LIMIT 1;
    """
    user_status_result = cur.execute(query_user_status, (user_id,)).fetchall()

    if not user_status_result:
        keyboard = InlineKeyboardMarkup()
        keyboard.add(
            InlineKeyboardButton("Пройти авторизацию", callback_data="authorization")
        )
        bot.send_message(
            message.chat.id,
            f"❔{message.from_user.first_name}, Вы здесь впервые и пока не авторизованы",
            reply_markup=keyboard,
        )
        user_authorization[user_id] = True

    elif user_status_result[0][2] == 0:
        keyboard = InlineKeyboardMarkup()
        keyboard.add(
            InlineKeyboardButton("Попробовать ещё раз", callback_data="authorization")
        )
        bot.send_message(
            message.chat.id,
            f"❓{message.from_user.first_name}, ранее Вы уже безуспешно пытались пройти авторизацию.",
            reply_markup=keyboard,
        )
        user_authorization[user_id] = True

    elif user_status_result[0][2] == 1:
        bot.delete_message(message.chat.id, message.message_id)
        bot.send_message(
            message.chat.id,
            f"""👋
Здравствуйте, {message.from_user.first_name}! Вы авторизованы в системе с {user_status_result[0][1]}.""",
        )
        # загрузка исходных данных и определение статуса
        bot.send_message(
            message.chat.id,
            f"""⏳Определяется статус исходных данных...""",
        )
        sticker = bot.send_sticker(
            message.chat.id,
            "CAACAgIAAxkBAAEM6bpm_mRva4ngeEpx0KVlCuwEUDsYVgACZgADTlzSKbUUiSuZ-b7XNgQ",
        )
        df_e, df_s, df_dm, user_status[user_id], fail[user_id] = datastatus(
            e_path, s_path, dm_path, dm_etalon_path, db_path
        )
        dhb_s[user_id] = dhb_status(df_e, df_s, db_path)
        status = user_status.get(user_id) + dhb_s.get(user_id)
        bot.delete_message(sticker.chat.id, sticker.id)
        bot.send_message(
            message.chat.id,
            f"""📂Статус исходных данных:
{status}""",
        )
        bot.delete_message(message.chat.id, message.message_id + 2)
        if not fail.get(user_id):
            keyboard = InlineKeyboardMarkup()
            keyboard.add(
                InlineKeyboardButton("Выполнить расчёты", callback_data="calculation")
            )
            bot.send_message(
                message.chat.id,
                f"❗️{message.from_user.first_name}, внимательно ознакомьтесь со статусом исходных данных. В них отсутствуют критические ошибки, препятствующие дальнейшей работе, но окончательное решение принимает аналитик.",
                reply_markup=keyboard,
            )
        elif fail.get(user_id):
            bot.send_message(
                message.chat.id,
                f"❗️{message.from_user.first_name}, это прискорбно, но состояние исходных данных не позволяет использовать их для расчётов.",
            )


@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    """авторизация неавторизованных: предлагается пин-код, засим одобрение Админом"""
    global user_id, transponder, question, answer, user_status_result, user_status, fail, accept, user_answer_id, dir_path, db_path, img_path, date_time, dhb_s, df_e, df_s, df_dm
    admin_id = admins(db_path)
    if call.data == "authorization" and user_authorization.get(call.from_user.id):
        bot.delete_message(call.message.chat.id, call.message.message_id)
        question, answer = transponder()
        user_id = call.from_user.id
        bot.send_message(
            call.message.chat.id,
            "".join([str(x) for x in question]),
        )
    elif call.data == "authorization" and not user_authorization.get(call.from_user.id):
        bot.delete_message(call.message.chat.id, call.message.message_id)
        user_id = call.from_user.id
        bot.send_message(
            call.message.chat.id,
            f"✅{call.from_user.first_name}, Вы были авторизованы в системе {user_status_result[0][1]}.",
        )
    elif call.data == "accept":
        # bot.delete_message(call.message.chat.id, call.message.message_id)
        # user_id = call.from_user.id
        accept = 1
        step_result, e = user_db(user_answer_id, date_time, accept)
        for a_id in admin_id:
            try:
                bot.delete_message(a_id, call.message.message_id)
            except Exception:
                pass
        if step_result:
            bot.send_message(
                user_answer_id,
                f"✅{call.from_user.first_name}, одобрение Вашего запроса получено. Для работы воспользуйтесть меню.",
            )
            bot.send_message(
                call.message.chat.id,
                f"✅{call.from_user.first_name}, новый пользователь зарегистрирован.",
            )
        elif not step_result:
            bot.send_message(
                user_answer_id,
                f"⛔️{call.from_user.first_name}, одобрение получено, но при внесении информации в базу данных произошёл сбой: {e}.",
            )
            bot.send_message(
                call.message.chat.id,
                f"⛔️При внесении информации в базу данных произошёл сбой: {e}.",
            )

        user_authorization[user_id] = False
    elif call.data == "reject":
        # bot.delete_message(call.message.chat.id, call.message.message_id)
        # user_id = call.from_user.id
        accept = 0
        step_result, e = user_db(user_answer_id, date_time, accept)
        for admin_id in admin_id:
            bot.delete_message(admin_id, call.message.message_id)
        bot.send_message(
            user_answer_id,
            f"⛔️{call.from_user.first_name}, запрос на авторизацию отколнён.",
        )
        user_authorization[user_id] = False
    # расчёты
    elif call.data == "calculation":
        bot.delete_message(call.message.chat.id, call.message.message_id)
        bot.send_message(
                call.message.chat.id,
                "⏳Производится расчёт федеральной рассылки",
            )
        time.sleep(1)
        calc_res, date_data = calc_filling_rus(df_e, df_s, df_dm, db_path)
        img_make(img_path, db_path, date_data)
        bot.delete_message(call.message.chat.id, call.message.message_id+1)
        bot.send_message(
                call.message.chat.id,
                f"🖼Созданы изображения для сообщений на {date_data}.",
            )        
        bot.send_message(
                call.message.chat.id,
                calc_res,
            )
        mes_rus_0, mes_rus_1, mes_rus_2, pdf_path = rus_messages(dir_path, db_path)
        id_chat = call.message.chat.id
        rus_send(id_chat, mes_rus_0, mes_rus_1, mes_rus_2, pdf_path)
        bot.send_message(
                call.message.chat.id,
                "⏳Производится расчёт региональной рассылки",
            )
        calc_res_sub, calc_res_mo = calc_filling_sub(df_e, df_s, df_dm, db_path) ### ТУТ
        time.sleep(1)
        bot.delete_message(call.message.chat.id, call.message.message_id+8)
        bot.send_message(
                call.message.chat.id,
                calc_res_sub,
            )
        bot.send_message(
                call.message.chat.id,
                calc_res_mo,
            )
        conn = sqlite3.connect(db_path)
        cur = conn.cursor()
        send_region_query = """
        SELECT rsd.reg_name
        FROM reg_send_data rsd
        WHERE rsd.send_flag = 1
        """
        send_region_list = [x[0] for x in cur.execute(send_region_query).fetchall()]
        reg_name = send_region_list[random.randint(0, len(send_region_list)-1)]
        id_send, mes_sub_0, mes_sub_1, mes_sub_2, mes_sub_3, sub_save_excel_path, pdf_path = sub_messages(dir_path, db_path, reg_name)
        id_send = call.message.chat.id
        bot.send_message(
                call.message.chat.id,
                f"🦠Пример рассылки для случайно выбранного региона (пускай это будет... {reg_name})",
            )
        time.sleep(3)
        sub_send(id_send, mes_sub_0, mes_sub_1, mes_sub_2, mes_sub_3, sub_save_excel_path, pdf_path, date_data, reg_name)
        sticker = bot.send_sticker(
            call.message.chat.id,
            "CAACAgIAAxkBAAEM8f9nB-ZOaAIzPawLqGwHXkPavAwjBQACHAADDbbSGbSyenEZW5bQNgQ",
        )


@bot.message_handler(content_types=["text"])
def handle_message(message):
    """обработка сообщений"""
    global messages, user_authorization, answer, user_db, user_status, fail, user_answer_id, date_time, dhb_s, db_password, edit_db
    admin_id = admins(db_path)
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    user_name = message.from_user.username
    user_first_name = message.from_user.first_name
    user_last_name = message.from_user.last_name
    messages[user_id] = message.text
    date_time = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    if user_authorization.get(user_id) and messages.get(user_id) == str(answer):
        # тут запрос админу, можно ли пропускать?
        user_answer_id = user_id
        keyboard = InlineKeyboardMarkup()
        keyboard.add(
            InlineKeyboardButton("Принять", callback_data="accept"),
            InlineKeyboardButton("Отклонить", callback_data="reject"),
        )
        for a_id in admin_id:
            try:
                bot.send_message(
                    a_id,
                    f"Пользователь {user_name} ({user_first_name} {user_last_name}, {user_id}) просит принять его.",
                    reply_markup=keyboard,
                )
            except Exception:
                pass
        bot.send_message(
            message.chat.id,
            f"✅Ответ верный, направлен запрос на авторизацию администратору чат-бота.",
        )
    elif user_authorization.get(user_id) and messages.get(user_id) != str(answer):
        step_result, e = user_db(user_id, date_time, 0)
        bot.delete_message(message.chat.id, message.message_id)
        bot.send_message(
            message.chat.id,
            f"⛔️Увы, {message.from_user.first_name}, ответ неверный. Для повторной попытки воспользуетесь меню.",
        )
        user_authorization[user_id] = False
    elif messages.get(user_id) == db_password:
        bot.delete_message(message.chat.id, message.message_id)
        bot.send_message(
            message.chat.id,
            f"""🗄
Вы можете направить один SQL-запрос для управления базой данных. Для следующего запроса требуется повторный ввод пароля.
    Структура таблиц:
admins:
    - admin_id
error_type:
    - final_state
    - error_type
    - error_type_dhb (классификация 'как на дашборде')
    - error_type_rus
regions:
    - reg_name_rus (эталонное название)
    - reg_name_lat  (эталонное название)
    - reg_name_cm  (как в ошибках и успехах)
    - reg_name_mc  (как в выгрузке витрин)
    - reg_name_pos  (как в выгрузках ПОС)
    - public_flag (отображаемость на дашборде (1/0))
user_status:
    - user_id
    - date_time
    - flag (1/0)
""",
        )
        user_db_edit[user_id] = True
    elif messages.get(user_id) and user_db_edit.get(user_id):
        q_result = edit_db(messages.get(user_id), db_path)
        try:
            bot.send_message(
                message.chat.id,
                q_result,
            )
        except Exception as e:
            bot.send_message(
                message.chat.id,
                f"Невозможно получить ответ: {e}",
            )
        user_db_edit[user_id] = False
# запуск бота
bot.infinity_polling()

2024-10-14 17:17:42,817 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2024-10-14 17:17:42,820 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


***